In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys; sys.path.insert(0, '..')

In [ ]:
import osmnx as ox
import pandas as pd

In [ ]:
ox.settings.useful_tags_way = ox.settings.useful_tags_way + ["cycleway"]

In [ ]:
# get network for a city
G = ox.graph_from_place(
    "Somerville, Massachusetts, USA",
    network_type="bike")

# convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
edges.crs

In [ ]:
G = ox.project_graph(G, to_crs="EPSG:26986")

In [ ]:
G = ox.consolidate_intersections(G, tolerance=10)

In [ ]:
edges["maxspeed_int"] = edges["maxspeed"].apply(extract_maxspeed).astype("Int64")

In [ ]:
edges['cycleway'].unique()

In [ ]:
# edges = edges.reset_index()

In [ ]:
edges['maxspeed'] = max_if_list(edges['maxspeed'])

In [ ]:
edges

In [ ]:
edges['cycleway'] = first_if_list(edges['cycleway'])

In [ ]:
max_if_list

In [ ]:
edges['cycleway'].unique()

In [ ]:
highways = edges.reset_index()['highway']

In [ ]:
unique = set()
for _, v in highways.items():
    if isinstance(v, list):
        unique.update(v)
    else:
        unique.add(v)
unique

In [ ]:
import numpy as np
import re

In [ ]:
edges[['highway', 'name', 'maxspeed']].sample(10)

In [ ]:
def extract_maxspeed(x):
    # Handle NaN early
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan

    # Handle lists and numpy arrays uniformly
    if isinstance(x, (list, np.ndarray)):
        speeds = [extract_maxspeed(i) for i in x]
        # filter out NaNs
        speeds = [s for s in speeds if not pd.isna(s)]
        return max(speeds) if speeds else np.nan

    # Convert to string and extract digits
    x_str = str(x)
    nums = re.findall(r'\d+', x_str)

    return int(max(nums, key=int)) if nums else np.nan

In [ ]:
edges['maxspeed_int'] = edges['maxspeed'].apply(extract_maxspeed)

In [ ]:
edges[['highway', 'name', 'maxspeed', 'maxspeed_int']].sample(10)

In [ ]:
edges.to_file("../data/somerville_roads.gpkg", layer="roads", driver="GPKG")

## Bike

In [ ]:
# Example: get driving network for a city
G = ox.graph_from_place("Somerville, Massachusetts, USA", network_type="bike")

In [ ]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

edges_drive.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")
edges_bike.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")